In [2]:
import numpy as np

import strawberryfields as sf
from strawberryfields.ops import *



In [3]:
boson_sampling = sf.Program(4)

In [4]:
with boson_sampling.context as q:
    # prepare the input fock states
    Fock(1) | q[0]
    Fock(1) | q[1]
    Vac     | q[2]
    Fock(1) | q[3]

    # rotation gates
    Rgate(0.5719)  | q[0]
    Rgate(-1.9782) | q[1]
    Rgate(2.0603)  | q[2]
    Rgate(0.0644)  | q[3]

    # beamsplitter array
    BSgate(0.7804, 0.8578)  | (q[0], q[1])
    BSgate(0.06406, 0.5165) | (q[2], q[3])
    BSgate(0.473, 0.1176)   | (q[1], q[2])
    BSgate(0.563, 0.1517)   | (q[0], q[1])
    BSgate(0.1323, 0.9946)  | (q[2], q[3])
    BSgate(0.311, 0.3231)   | (q[1], q[2])
    BSgate(0.4348, 0.0798)  | (q[0], q[1])
    BSgate(0.4368, 0.6157)  | (q[2], q[3])

In [6]:
eng = sf.Engine(backend="fock", backend_options={"cutoff_dim": 7})


In [7]:
results = eng.run(boson_sampling)


In [8]:
# extract the joint Fock probabilities
probs = results.state.all_fock_probs()

# print the joint Fock state probabilities
print(probs[1, 1, 0, 1])
print(probs[2, 0, 0, 1])

0.17468916048563934
0.10644192724642337


In [9]:
import numpy as np
from numpy.linalg import multi_dot
from scipy.linalg import block_diag

In [10]:
Uphase = np.diag([np.exp(0.5719*1j),np.exp(-1.9782*1j),np.exp(2.0603*1j),np.exp(0.0644*1j)])


In [11]:
BSargs = [
    (0.7804, 0.8578),
    (0.06406, 0.5165),
    (0.473, 0.1176),
    (0.563, 0.1517),
    (0.1323, 0.9946),
    (0.311, 0.3231),
    (0.4348, 0.0798),
    (0.4368, 0.6157)
]

In [12]:
t_r_amplitudes = [(np.cos(q), np.exp(p*1j)*np.sin(q)) for q,p in BSargs]


In [13]:
BSunitaries = [np.array([[t, -np.conj(r)], [r, t]]) for t,r in t_r_amplitudes]


In [14]:
UBS1 = block_diag(*BSunitaries[0:2])
UBS2 = block_diag([[1]], BSunitaries[2], [[1]])
UBS3 = block_diag(*BSunitaries[3:5])
UBS4 = block_diag([[1]], BSunitaries[5], [[1]])
UBS5 = block_diag(*BSunitaries[6:8])

In [15]:
U = multi_dot([UBS5, UBS4, UBS3, UBS2, UBS1, Uphase])
print(np.round(U, 4))

[[ 0.2195-0.2565j  0.6111+0.5242j -0.1027+0.4745j -0.0273+0.0373j]
 [ 0.4513+0.6026j  0.457 +0.0123j  0.1316-0.4504j  0.0353-0.0532j]
 [ 0.0387+0.4927j -0.0192-0.3218j -0.2408+0.5244j -0.4584+0.3296j]
 [-0.1566+0.2246j  0.11  -0.1638j -0.4212+0.1836j  0.8188+0.068j ]]


In [16]:
prog_unitary = sf.Program(4)
prog_unitary.circuit = boson_sampling.circuit[4:]
prog_compiled = prog_unitary.compile(compiler="gaussian_unitary")

In [17]:
prog_compiled.print()


GaussianTransform([[ 0.2195  0.6111 -0.1027 -0.0273  0.2565 -0.5242 -0.4745 -0.0373]
 [ 0.4513  0.457   0.1316  0.0353 -0.6026 -0.0123  0.4504  0.0532]
 [ 0.0387 -0.0192 -0.2408 -0.4584 -0.4927  0.3218 -0.5244 -0.3296]
 [-0.1566  0.11   -0.4212  0.8188 -0.2246  0.1638 -0.1836 -0.068 ]
 [-0.2565  0.5242  0.4745  0.0373  0.2195  0.6111 -0.1027 -0.0273]
 [ 0.6026  0.0123 -0.4504 -0.0532  0.4513  0.457   0.1316  0.0353]
 [ 0.4927 -0.3218  0.5244  0.3296  0.0387 -0.0192 -0.2408 -0.4584]
 [ 0.2246 -0.1638  0.1836  0.068  -0.1566  0.11   -0.4212  0.8188]]) | (q[0], q[1], q[2], q[3])


In [18]:
S = prog_compiled.circuit[0].op.p[0]
U = S[:4, :4] + 1j*S[4:, :4]
print(U)

[[ 0.2195-0.2565j  0.6111+0.5242j -0.1027+0.4745j -0.0273+0.0373j]
 [ 0.4513+0.6026j  0.457 +0.0123j  0.1316-0.4504j  0.0353-0.0532j]
 [ 0.0387+0.4927j -0.0192-0.3218j -0.2408+0.5244j -0.4584+0.3296j]
 [-0.1566+0.2246j  0.11  -0.1638j -0.4212+0.1836j  0.8188+0.068j ]]


In [19]:
boson_sampling = sf.Program(4)

with boson_sampling.context as q:
    # prepare the input fock states
    Fock(1) | q[0]
    Fock(1) | q[1]
    Vac     | q[2]
    Fock(1) | q[3]

    Interferometer(U) | q

In [20]:
boson_sampling.compile(compiler="fock").print()


Fock(1) | (q[0])
Fock(1) | (q[1])
Vac | (q[2])
Fock(1) | (q[3])
Rgate(-3.124) | (q[0])
BSgate(0.9465, 0) | (q[0], q[1])
Rgate(2.724) | (q[2])
BSgate(0.09485, 0) | (q[2], q[3])
Rgate(-0.9705) | (q[1])
BSgate(0.7263, 0) | (q[1], q[2])
Rgate(-1.788) | (q[0])
BSgate(0.8246, 0) | (q[0], q[1])
Rgate(5.343) | (q[0])
Rgate(2.93) | (q[1])
Rgate(3.133) | (q[2])
Rgate(0.07904) | (q[3])
BSgate(-0.533, 0) | (q[2], q[3])
Rgate(2.45) | (q[2])
BSgate(-0.03962, 0) | (q[1], q[2])
Rgate(2.508) | (q[1])


In [21]:
print(probs[2,0,0,1])


0.10644192724642337


In [22]:
from thewalrus import perm


In [23]:
U[:,[0,1,3]]


array([[ 0.2195-0.2565j,  0.6111+0.5242j, -0.0273+0.0373j],
       [ 0.4513+0.6026j,  0.457 +0.0123j,  0.0353-0.0532j],
       [ 0.0387+0.4927j, -0.0192-0.3218j, -0.4584+0.3296j],
       [-0.1566+0.2246j,  0.11  -0.1638j,  0.8188+0.068j ]])

In [24]:
U[:,[0,1,3]][[0,0,3]]


array([[ 0.2195-0.2565j,  0.6111+0.5242j, -0.0273+0.0373j],
       [ 0.2195-0.2565j,  0.6111+0.5242j, -0.0273+0.0373j],
       [-0.1566+0.2246j,  0.11  -0.1638j,  0.8188+0.068j ]])

In [25]:
print(np.abs(perm(U[:, [0,1,3]][[0,0,3]]))**2 / 2)


0.10644192724642332


In [26]:
BS = np.abs(perm(U[:, [0,1,3]][[0,0,3]]))**2 / 2
SF = probs[2,0,0,1]

print(100*np.abs(BS-SF)/BS)

5.2151584124124477e-14


In [27]:
print(probs[3,0,0,0])
print(np.abs(perm(U[:, [0,1,3]][[0,0,0]]))**2 / 6)


0.000945848334713249
0.0009458483347132489


In [28]:
print(probs[1,1,0,1])
print(np.abs(perm(U[:, [0,1,3]][[0,1,3]]))**2 / 1)

0.17468916048563934
0.17468916048563943
